# Projet d'optimisation 
## Par  Rania Fathi et Pierre Sion 

## I- Modélisation 

### Question 1 :
On définit l'élasticité-prix comme étant la variation relative de la quantité demandée par rapport au prix, soit encore: 
    $\varepsilon_i = \frac{\frac{dx_i}{x_i}}{\frac{dp_i}{p_i}} $ 
où l'on a noté $x_i$ la demande et $p_i$ le prix du ième produit 
    
Cela peut encore s'écrire: 
$\varepsilon_i = \frac{\partial x_i}{\partial p_i} \frac{p_i}{x_i}$

Comme on s'attend à ce que la demande pour un produit diminue si le prix augmente, on a $\varepsilon_i \leq 0$

Pour ce qui est de l'élasticité prix croisée entre le gouda et l'edam, elle vaut: 

$\varepsilon_c = \frac{\frac{dx_g}{x_g}}{\frac{dp_e}{p_e}} = \frac{\partial x_g}{\partial p_e} \frac{p_e}{x_g}$

Les biens considérés étant substituables, on a $\varepsilon_c \geq 0$ 

### Question 2 :
Le lait brut possède une quantité de matière grasse totale répartie entre les quatre produits, que l'on note $M$. En notant $m_i$ la teneur en matière grasse présente dans chaque produit, on a : 

$ \sum_{i=1}^{4}m_i x_i \leq M$

On fait par ailleurs le choix de mettre une inégalité plutôt qu'une égalité pour considérer le cas où tout le lait n'est pas transformé 

### Question 3 : 
De la même manière, on note $L$ la quantité totale de lactose contenue dans le lait, et $l_i$ celle contenue dans chaque produit, et on a alors la contrainte suivante: 

$\sum_{i=1}^{4} l_i x_i \leq L $

### Question 4 :

Afin de garantir la paix sociale, la moyenne (pondérée par la part de chaque produit dans le budget) des changements de prix relatifs ne doit pas être positive.
On note $b_i$ la part du ième produit dans le budget (déterminée à partir des données de l'année N-1). Cette contrainte s'écrit donc: 
$\sum_{i=1}^{4}b_i \frac{dp_i}{p_i} \leq 0$

Comme les contraintes dont on dispose jusque maintenant portent sur les quantités et non pas les prix, on la réécrit sous la forme suivante : $ \sum_{i=1}^4 \frac{b_i}{\epsilon_i} \frac{dx_i}{x_i} \leq 0$ 

### Question 5 :

_En supposant qu'on dispose déjà des $(p_i)_{1\leq i\leq4}$, $(b_i)_{1\leq i\leq4}$, $(\epsilon_i)_{1\leq i\leq4}$, $(m_i)_{1\leq i \leq 4}$, de $M$ et $L$_  
On dispose donc du problème d'optimisation sous contraintes suivant : $\max_{c(x) \leq 0} \Pi (x)$ 

où le profit $\Pi:\mathbf{R}^4 \rightarrow \mathbf{R}  $ vaut $\Pi(x) = \sum_{i=1}^4 p_i x_i $ (En toute rigueur, $\Pi$ est égal au revenu plutôt qu'au profit, mais l'omission du terme $-C$ correspondant au coût total du lait acheté ne modifie pas le problème d'optimisation à résoudre).

Et les contraintes $c : \mathbf{R}^4 \rightarrow \mathbf{R}^7$ sont les suivantes : 

$c_1(x) = - x_1 \\ 
c_2(x) = - x_2\\ 
c_3(x) = - x_3\\ 
c_4(x) = - x_4$

Ces quatre premières contraintes traduisent le fait que les quantités considérées sont nécessairement positives. Et nous disposons des deux contraintes des questions 2 et 3: 

$c_5(x) = \sum_{i=1}^{4}m_i x_i - M\\ 
c_6(x) = \sum_{i=1}^{4} l_i x_i - L $

Et enfin :
$c_7(x) =  \sum_{i=1}^4 \frac{b_i}{\epsilon_i} \frac{dx_i}{x_i} \leq 0$

NB: Il aurait été possible d'ajouter les contraintes portant sur le signe de l'élasticité des produits, imposant que l'augmentation du prix d'un produit réduit la demande, ou encore celles stipulant que les prix des produits se doivent d'être positifs 

## II- Implémentation 

### Cadre de travail

In [6]:
import numpy as np


A = np.array([[3.0825,0,0,0],
              [0,0.0405,0,0,],
              [0,0,0.0271,-0.0031],
              [0,0,-0.0031,0.0054]])
b = np.array([2671,135,103,19])

C = np.array([[-0.0401,-0.0162,-0.0039,0.0002],
              [-0.1326,-0.0004,-0.0034,0.0006],
              [1.5413,0,0,0],
              [0,0.0203,0,0],
              [0,0,0.0136,-0.0015],
              [0,0,-0.0016,0.0027],
              [0.0160,0.0004,0.0005,0.0002]])

d = np.array([-92.6,-29,2671,135,103,19,10])

### Première méthode : Algorithme de Arrow-Hurwickz

On commence par construire le projecteur sur $\mathbb{R}_{+}$ :

In [7]:
def proj(x):
    for i in range(0,len(x)):
        if x[i]<0:
            x[i]=0
    return x

Ce qui permet d'implémenter l'algorithme AH : 

In [8]:
P0=np.array([2055,54,63,17])

LAMBDA=np.array([1,1,1,1,1,1,1])

def algo_ah(p0=P0,lambda0=LAMBDA,epsilon=0.01,mu=10**-5,alpha=0.001):   
    l=lambda0
    pl=p0
    pk=pl-epsilon*(b-np.dot(A,pl)+(np.transpose(C)).dot(l))
    while(np.linalg.norm(pk-pl)>mu):
        pl=pk
        pk=pk-epsilon*(b-np.dot(A,pk)+(np.transpose(C)).dot(l))
        l=proj(l+alpha*(np.dot(C,pk)-d))
        print(pk)
    return pk

In [9]:
print(algo_ah())

[2129.37191663   51.34312009   60.97271926   16.6179071 ]
[2168.27620497   50.01390013   59.95868081   16.42689276]
[2208.36903125   48.68416865   58.94437832   16.23590001]
[2249.68607191   47.35392517   57.92981149   16.0449289 ]
[2292.26413463   46.02316937   56.91498156   15.85397919]
[2336.1413747    44.69190168   55.89989598   15.66304956]
[2381.35679612   43.36012162   54.88455461   15.47214002]
[2427.95057335   42.02782867   53.86895733   15.28125058]
[2475.96408646   40.69500765   52.85310397   15.09038124]
[2525.43995732   39.3616455    51.83699441   14.89953203]
[2576.42208685   38.02774167   50.82062541   14.70870328]
[2628.95569338   36.69329562   49.80398598   14.51789622]
[2683.08735225   35.35830679   48.78707596   14.32711084]
[2738.86503645   34.0227746    47.76989519   14.13634716]
[2796.33815864   32.68669847   46.75244351   13.9456052 ]
[2855.5576143    31.35007782   45.73472074   13.75488496]
[2916.57582623   30.01291203   44.71672671   13.56418646]
[2979.44679042

[ 1.79547305e+14 -1.33237292e+07 -1.65720458e+07 -6.56613970e+06]
[ 1.84935437e+14 -1.37235302e+07 -1.70693387e+07 -6.76318250e+06]
[ 1.90485264e+14 -1.41353290e+07 -1.75815551e+07 -6.96613845e+06]
[ 1.96201639e+14 -1.45594856e+07 -1.81091428e+07 -7.17518502e+06]
[ 2.02089560e+14 -1.49963709e+07 -1.86525632e+07 -7.39050497e+06]
[ 2.08154175e+14 -1.54463668e+07 -1.92122913e+07 -7.61228657e+06]
[ 2.14400786e+14 -1.59098669e+07 -1.97888165e+07 -7.84072372e+06]
[ 2.20834854e+14 -1.63872763e+07 -2.03826430e+07 -8.07601617e+06]
[ 2.27462007e+14 -1.68790126e+07 -2.09942899e+07 -8.31836962e+06]
[ 2.34288037e+14 -1.73855055e+07 -2.16242920e+07 -8.56799599e+06]
[ 2.41318913e+14 -1.79071980e+07 -2.22732002e+07 -8.82511353e+06]
[ 2.48560782e+14 -1.84445462e+07 -2.29415817e+07 -9.08994704e+06]
[ 2.56019977e+14 -1.89980200e+07 -2.36300210e+07 -9.36272807e+06]
[ 2.63703018e+14 -1.95681031e+07 -2.43391200e+07 -9.64369514e+06]
[ 2.71616624e+14 -2.01552942e+07 -2.50694988e+07 -9.93309389e+06]
[ 2.797677

[ 1.75005497e+23 -1.29853983e+16 -1.61519371e+16 -6.39990698e+15]
[ 1.80257331e+23 -1.33750841e+16 -1.66366493e+16 -6.59196524e+15]
[ 1.85666771e+23 -1.37764642e+16 -1.71359075e+16 -6.78978708e+15]
[ 1.91238545e+23 -1.41898896e+16 -1.76501482e+16 -6.99354546e+15]
[ 1.96977525e+23 -1.46157216e+16 -1.81798210e+16 -7.20341854e+15]
[ 2.02888730e+23 -1.50543327e+16 -1.87253890e+16 -7.41958981e+15]
[ 2.08977328e+23 -1.55061063e+16 -1.92873293e+16 -7.64224828e+15]
[ 2.15248641e+23 -1.59714374e+16 -1.98661332e+16 -7.87158863e+15]
[ 2.21708153e+23 -1.64507329e+16 -2.04623067e+16 -8.10781138e+15]
[ 2.28361513e+23 -1.69444118e+16 -2.10763711e+16 -8.35112306e+15]
[ 2.35214537e+23 -1.74529058e+16 -2.17088633e+16 -8.60173641e+15]
[ 2.42273217e+23 -1.79766594e+16 -2.23603362e+16 -8.85987056e+15]
[ 2.49543724e+23 -1.85161307e+16 -2.30313596e+16 -9.12575119e+15]
[ 2.57032416e+23 -1.90717912e+16 -2.37225201e+16 -9.39961078e+15]
[ 2.64745841e+23 -1.96441269e+16 -2.44344220e+16 -9.68168877e+15]
[ 2.726907

[ 3.28132445e+33 -2.43474095e+26 -3.02846179e+26 -1.19997209e+26]
[ 3.37979548e+33 -2.50780640e+26 -3.11934453e+26 -1.23598270e+26]
[ 3.48122159e+33 -2.58306451e+26 -3.21295463e+26 -1.27307397e+26]
[ 3.58569144e+33 -2.66058109e+26 -3.30937391e+26 -1.31127834e+26]
[ 3.69329639e+33 -2.74042390e+26 -3.40868670e+26 -1.35062920e+26]
[ 3.80413051e+33 -2.82266276e+26 -3.51097982e+26 -1.39116096e+26]
[ 3.91829072e+33 -2.90736957e+26 -3.61634270e+26 -1.43290905e+26]
[ 4.03587682e+33 -2.99461839e+26 -3.72486748e+26 -1.47591000e+26]
[ 4.15699162e+33 -3.08448551e+26 -3.83664904e+26 -1.52020137e+26]
[ 4.28174102e+33 -3.17704950e+26 -3.95178511e+26 -1.56582192e+26]
[ 4.41023410e+33 -3.27239129e+26 -4.07037636e+26 -1.61281151e+26]
[ 4.54258319e+33 -3.37059424e+26 -4.19252648e+26 -1.66121124e+26]
[ 4.67890402e+33 -3.47174422e+26 -4.31834227e+26 -1.71106343e+26]
[ 4.81931577e+33 -3.57592967e+26 -4.44793373e+26 -1.76241165e+26]
[ 4.96394122e+33 -3.68324167e+26 -4.58141417e+26 -1.81530081e+26]
[ 5.112906

[ 2.24298831e+42 -1.66429610e+35 -2.07014104e+35 -8.20255181e+34]
[ 2.31029936e+42 -1.71424086e+35 -2.13226502e+35 -8.44870661e+34]
[ 2.37963038e+42 -1.76568444e+35 -2.19625331e+35 -8.70224840e+34]
[ 2.45104199e+42 -1.81867182e+35 -2.26216186e+35 -8.96339887e+34]
[ 2.52459663e+42 -1.87324932e+35 -2.33004829e+35 -9.23238634e+34]
[ 2.60035861e+42 -1.92946467e+35 -2.39997197e+35 -9.50944600e+34]
[ 2.67839417e+42 -1.98736701e+35 -2.47199402e+35 -9.79482009e+34]
[ 2.75877155e+42 -2.04700698e+35 -2.54617742e+35 -1.00887581e+35]
[ 2.84156101e+42 -2.10843672e+35 -2.62258704e+35 -1.03915171e+35]
[ 2.92683495e+42 -2.17170993e+35 -2.70128966e+35 -1.07033618e+35]
[ 3.01466792e+42 -2.23688195e+35 -2.78235412e+35 -1.10245647e+35]
[ 3.10513671e+42 -2.30400974e+35 -2.86585129e+35 -1.13554068e+35]
[ 3.19832043e+42 -2.37315201e+35 -2.95185416e+35 -1.16961773e+35]
[ 3.29430056e+42 -2.44436921e+35 -3.04043795e+35 -1.20471742e+35]
[ 3.39316100e+42 -2.51772361e+35 -3.13168009e+35 -1.24087044e+35]
[ 3.494988

...

[ 1.40629750e+266 -1.04347198e+259 -1.29792659e+259 -5.14279456e+258]
[ 1.44849984e+266 -1.07478609e+259 -1.33687677e+259 -5.29712746e+258]
[ 1.49196865e+266 -1.10703992e+259 -1.37699582e+259 -5.45609181e+258]
[ 1.53674194e+266 -1.14026168e+259 -1.41831883e+259 -5.61982661e+258]
[ 1.58285886e+266 -1.17448041e+259 -1.46088193e+259 -5.78847502e+258]
[ 1.63035973e+266 -1.20972603e+259 -1.50472232e+259 -5.96218449e+258]
[ 1.67928607e+266 -1.24602935e+259 -1.54987834e+259 -6.14110690e+258]
[ 1.72968067e+266 -1.28342211e+259 -1.59638948e+259 -6.32539869e+258]
[ 1.78158759e+266 -1.32193702e+259 -1.64429639e+259 -6.51522099e+258]
[ 1.83505221e+266 -1.36160774e+259 -1.69364097e+259 -6.71073977e+258]
[ 1.89012129e+266 -1.40246896e+259 -1.74446635e+259 -6.91212597e+258]
[ 1.94684295e+266 -1.44455641e+259 -1.79681699e+259 -7.11955569e+258]
[ 2.00526681e+266 -1.48790688e+259 -1.85073864e+259 -7.33321028e+258]
[ 2.06544395e+266 -1.53255828e+259 -1.90627845e+259 -7.55327654e+258]
[ 2.12742697e+266 -1

[ 2.40399574e+275 -1.78376352e+268 -2.21874105e+268 -8.79135191e+267]
[ 2.47613854e+275 -1.83729344e+268 -2.28532444e+268 -9.05517633e+267]
[ 2.55044632e+275 -1.89242977e+268 -2.35390598e+268 -9.32691800e+267]
[ 2.62698404e+275 -1.94922072e+268 -2.42454561e+268 -9.60681451e+267]
[ 2.70581862e+275 -2.00771594e+268 -2.49730511e+268 -9.89511059e+267]
[ 2.78701899e+275 -2.06796657e+268 -2.57224808e+268 -1.01920583e+268]
[ 2.87065614e+275 -2.13002529e+268 -2.64944006e+268 -1.04979173e+268]
[ 2.95680321e+275 -2.19394637e+268 -2.72894854e+268 -1.08129549e+268]
[ 3.04553551e+275 -2.25978569e+268 -2.81084303e+268 -1.11374467e+268]
[ 3.13693063e+275 -2.32760081e+268 -2.89519513e+268 -1.14716764e+268]
[ 3.23106847e+275 -2.39745104e+268 -2.98207860e+268 -1.18159361e+268]
[ 3.32803135e+275 -2.46939744e+268 -3.07156941e+268 -1.21705269e+268]
[ 3.42790404e+275 -2.54350292e+268 -3.16374579e+268 -1.25357588e+268]
[ 3.53077386e+275 -2.61983227e+268 -3.25868834e+268 -1.29119511e+268]
[ 3.63673075e+275 -2

[ 7.15384231e+282 -5.30814708e+275 -6.60255895e+275 -2.61614213e+275]
[ 7.36852582e+282 -5.46744213e+275 -6.80069870e+275 -2.69465135e+275]
[ 7.58965189e+282 -5.63151755e+275 -7.00478454e+275 -2.77551660e+275]
[ 7.81741384e+282 -5.80051680e+275 -7.21499490e+275 -2.85880857e+275]
[ 8.05201083e+282 -5.97458764e+275 -7.43151357e+275 -2.94460010e+275]
[ 8.29364797e+282 -6.15388226e+275 -7.65452987e+275 -3.03296619e+275]
[ 8.54253652e+282 -6.33855743e+275 -7.88423878e+275 -3.12398411e+275]
[ 8.79889410e+282 -6.52877462e+275 -8.12084115e+275 -3.21773343e+275]
[ 9.06294486e+282 -6.72470013e+275 -8.36454385e+275 -3.31429613e+275]
[ 9.33491966e+282 -6.92650529e+275 -8.61555996e+275 -3.41375663e+275]
[ 9.61505630e+282 -7.13436652e+275 -8.87410895e+275 -3.51620189e+275]
[ 9.90359971e+282 -7.34846557e+275 -9.14041687e+275 -3.62172149e+275]
[ 1.02008022e+283 -7.56898964e+275 -9.41471656e+275 -3.73040768e+275]
[ 1.05069235e+283 -7.79613153e+275 -9.69724787e+275 -3.84235550e+275]
[ 1.08222315e+283 -8

[ 1.54926542e+292 -1.14955409e+285 -1.42987723e+285 -5.66562464e+284]
[ 1.59575816e+292 -1.18405168e+285 -1.47278719e+285 -5.83564743e+284]
[ 1.64364613e+292 -1.21958453e+285 -1.51698486e+285 -6.01077252e+284]
[ 1.69297119e+292 -1.25618370e+285 -1.56250887e+285 -6.19115303e+284]
[ 1.74377647e+292 -1.29388119e+285 -1.60939904e+285 -6.37694668e+284]
[ 1.79610640e+292 -1.33270997e+285 -1.65769637e+285 -6.56831592e+284]
[ 1.85000673e+292 -1.37270398e+285 -1.70744307e+285 -6.76542805e+284]
[ 1.90552458e+292 -1.41389820e+285 -1.75868265e+285 -6.96845543e+284]
[ 1.96270849e+292 -1.45632863e+285 -1.81145991e+285 -7.17757556e+284]
[ 2.02160847e+292 -1.50003238e+285 -1.86582098e+285 -7.39297130e+284]
[ 2.08227601e+292 -1.54504766e+285 -1.92181341e+285 -7.61483096e+284]
[ 2.14476415e+292 -1.59141383e+285 -1.97948615e+285 -7.84334853e+284]
[ 2.20912754e+292 -1.63917142e+285 -2.03888961e+285 -8.07872381e+284]
[ 2.27542244e+292 -1.68836220e+285 -2.10007575e+285 -8.32116259e+284]
[ 2.34370681e+292 -1

[ 1.06296978e+302 -7.88722996e+294 -9.81056100e+294 -3.88725374e+294]
[ 1.09486901e+302 -8.12392210e+294 -1.01049714e+295 -4.00390843e+294]
[ 1.12772553e+302 -8.36771726e+294 -1.04082170e+295 -4.12406388e+294]
[ 1.16156805e+302 -8.61882859e+294 -1.07205627e+295 -4.24782513e+294]
[ 1.19642617e+302 -8.87747567e+294 -1.10422819e+295 -4.37530041e+294]
[ 1.23233037e+302 -9.14388462e+294 -1.13736557e+295 -4.50660116e+294]
[ 1.26931204e+302 -9.41828839e+294 -1.17149739e+295 -4.64184218e+294]
[ 1.30740351e+302 -9.70092688e+294 -1.20665348e+295 -4.78114173e+294]
[ 1.34663808e+302 -9.99204723e+294 -1.24286460e+295 -4.92462159e+294]
[ 1.38705007e+302 -1.02919040e+295 -1.28016239e+295 -5.07240721e+294]
[ 1.42867481e+302 -1.06007593e+295 -1.31857947e+295 -5.22462782e+294]
[ 1.47154868e+302 -1.09188832e+295 -1.35814944e+295 -5.38141649e+294]
[ 1.51570918e+302 -1.12465538e+295 -1.39890688e+295 -5.54291032e+294]
[ 1.56119491e+302 -1.15840577e+295 -1.44088743e+295 -5.70925050e+294]
[ 1.60804565e+302 -1